In [24]:
import pandas as pd

file_path = 'data/crm.xlsx'
crm = pd.read_excel(file_path,parse_dates=['DT'],index_col='DT')
crm.columns =["sample","ni","fe","co","alu","sil","phos","sul","mag","cal","chrom","man","ti","xrf", "note"]

In [28]:
crm_features = ['ni', 'fe', 'co', 'alu', 'sil', 'phos', 'sul', 'mag']
crm_daily = crm.groupby(['xrf','sample'])[crm_features].resample('D').mean().dropna().round(2).reset_index()

In [29]:
crm_daily.head()

,xrf,sample,DT,ni,fe,alu,sil,mag
0,2,S1,2021-06-06,0.63,48.21,10.35,1.75,0.98
1,2,S1,2021-06-30,0.64,48.96,10.66,1.68,0.91
2,2,S1,2021-07-01,0.63,48.63,10.25,1.76,0.97
3,2,S1,2021-07-02,0.63,48.64,10.20,1.80,1.00
4,2,S1,2021-07-03,0.63,48.74,10.19,1.83,1.04


In [30]:
s1 = crm_daily[crm_daily['sample']=='S1']
s1.head()

,xrf,sample,DT,ni,fe,alu,sil,mag
0,2,S1,2021-06-06,0.63,48.21,10.35,1.75,0.98
1,2,S1,2021-06-30,0.64,48.96,10.66,1.68,0.91
2,2,S1,2021-07-01,0.63,48.63,10.25,1.76,0.97
3,2,S1,2021-07-02,0.63,48.64,10.20,1.80,1.00
4,2,S1,2021-07-03,0.63,48.74,10.19,1.83,1.04


In [33]:
y=s1.ni
crm_features = ['xrf','fe', 'alu', 'sil','mag']
X=s1[crm_features]

In [34]:
X.head()

,xrf,fe,alu,sil,mag
0,2,48.21,10.35,1.75,0.98
1,2,48.96,10.66,1.68,0.91
2,2,48.63,10.25,1.76,0.97
3,2,48.64,10.20,1.80,1.00
4,2,48.74,10.19,1.83,1.04


In [35]:
from sklearn.tree import DecisionTreeRegressor

#define model. specify a number for random state to ensure some results each run
s1_model = DecisionTreeRegressor(random_state=1)

#fit model
s1_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [36]:
print("Making predictions for the following samples:")
print(X.head())
print("The predictions are")
print(s1_model.predict(X.head()))

Making predictions for the following samples:
   xrf     fe    alu   sil   mag
0    2  48.21  10.35  1.75  0.98
1    2  48.96  10.66  1.68  0.91
2    2  48.63  10.25  1.76  0.97
3    2  48.64  10.20  1.80  1.00
4    2  48.74  10.19  1.83  1.04
The predictions are
[0.63 0.64 0.63 0.63 0.63]


In [37]:
#calculate the mean absolute error
from sklearn.metrics import mean_absolute_error

predicted_ni = s1_model.predict(X)
mean_absolute_error(y, predicted_ni)


2.463412790201715e-17

In [38]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
s1_model = DecisionTreeRegressor()
# Fit model
s1_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = s1_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.005333333333333371
